# 03 - CI/CD Pipelines

## Install Required Packages

In [2]:
!pip install pyathena
!pip install -U sagemaker

## Import Required Libraries

In [3]:
import boto3 # aws sdk for python
import sagemaker # machine learning platform
from sagemaker.workflow.pipeline_context import PipelineSession
import sys

import json # encoder and decoder
import numpy as np # array manipulation
import os # operating system interfaces
import pandas as pd # python data analysis
import re # regular expressions
from pyathena import connect # athena client
from sagemaker.pytorch.estimator import PyTorch # PyTorch estimator
from sagemaker.pytorch.model import PyTorchModel # PyTorch model
from time import gmtime, strftime, sleep # time-related functions

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Set Up Environment

In [4]:
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

pipeline_session = PipelineSession()

model_package_group_name = f"SafetyModelPackageGroupName"

# define prefixes for the safety catalog and data directories
prefix_catalog = 'safety/catalog'
prefix_data = 'safety/data'

# print s3 locations
#print('Data directory location:', f"s3://{default_bucket}/{prefix_data}")

## Create Input Data Using Catalog

In [5]:
!mkdir -p data

In [6]:
# define database name
database_name = 'safetydb'

# define table name
table_name_csv = 'catalog_csv'

# set s3 temporary staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(default_bucket)

# define connection parameters
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# define sql query statement
statement = """SELECT * FROM {}.{}
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 100""".format(
    database_name, table_name_csv
)

# print sql statement for review before executing
print('SQL query SELECT statement:\n', statement)

SQL query SELECT statement:
 SELECT * FROM safetydb.catalog_csv
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 100


In [24]:
# execute sql query and display results
df_catalog_query = pd.read_sql(statement, conn)
#df_catalog_query.head(10)

df_catalog_query.to_csv(f"data/catalog_query.csv")

/tmp/ipykernel_34/1054740236.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_catalog_query = pd.read_sql(statement, conn)


## Create Pipeline

### Define Input Data

In [7]:
local_path = "data/catalog_query.csv"

s3 = boto3.resource("s3")

catalog_uri = f"s3://{default_bucket}/{prefix_catalog}"

input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=catalog_uri,
)

print(input_data_uri)

s3://sagemaker-us-east-1-414754026690/safety/catalog/catalog_query.csv


### Define Batch Data

In [8]:
batch_data_uri = f"s3://{default_bucket}/{prefix_data}/split_cicd/batch/images"

print(batch_data_uri)

s3://sagemaker-us-east-1-414754026690/safety/data/split_cicd/batch/images


## Define Pipeline Parameters

In [143]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.r5.xlarge" # memory optimized
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)

map_threshold = ParameterFloat(name="mAPThreshold", default_value=0.001)

## Define a Processing Step for Feature Engineering

In [10]:
!mkdir -p code

In [11]:
%%writefile code/preprocessing.py
import argparse
#import os
#import requests
#import tempfile
import numpy as np
import pandas as pd
#from sklearn.pipeline import Pipeline
import subprocess

# define function to copy files for respective data splits to corresponding s3 destinations
# provide 'split_name' as either 'train', 'val', 'test', or 'batch'
# provide 'split_list' as either 'train_list', 'val_list', 'test_list', or 'batch_list'
def split_dataset(split_name, split_list):
    
    # counter for printing copy progress
    num_files_copied = 0

    # iterate through each sample in split
    for index, sample in data[split_list].iterrows():

        # source/destination variables for individual sample
        cp_image_source = f"{s3_images_source}{sample['img_filename']}"
        cp_image_dest = f"{s3_split_dest}{split_name}/images/"
        cp_label_source = f"{s3_labels_source}{sample['label_filename']}"
        cp_label_dest = f"{s3_split_dest}{split_name}/labels/"

        # copy from source to destination
        #!aws s3 cp $cp_image_source $cp_image_dest --exclude "*" --include "*.jpg" --only-show-errors
        #!aws s3 cp $cp_label_source $cp_label_dest --exclude "*" --include "*.txt" --only-show-errors
        """
        subprocess.run(f"aws s3 cp {cp_image_source} {cp_image_dest} --exclude '*' --include '*.jpg' --only-show-errors", shell=True)
        subprocess.run(f"aws s3 cp {cp_label_source} {cp_label_dest} --exclude '*' --include '*.txt' --only-show-errors", shell=True)
        
        # increment counter
        num_files_copied += 1

        # print status after every 500 files copied
        if num_files_copied % 10 == 0:
            print(f"{num_files_copied} files copied.")
        """


if __name__ == "__main__":
    
    parser = argparse.ArgumentParser()

    parser.add_argument('--s3-images-source', type=str)
    parser.add_argument('--s3-labels-source', type=str)
    parser.add_argument('--s3-split-dest', type=str)
    
    # obtain arguments
    args, _ = parser.parse_known_args()
    
    base_dir = "/opt/ml/processing"
    
    data = pd.read_csv(
        f"{base_dir}/input/catalog_query.csv"
    )
    
    # data split in four sets - training, validation, test, and batch inference
    rand_split = np.random.rand(len(data))
    train_list = rand_split < 0.4
    val_list = (rand_split >= 0.4) & (rand_split < 0.5)
    test_list = (rand_split >= 0.5) & (rand_split < 0.6)
    batch_list = rand_split >= 0.6 # "production" data

    # print data splits
    print('Data Splits:')
    print('------------')
    print(f"Train :   {sum(train_list)} samples")
    print(f"Val   :   {sum(val_list)} samples")
    print(f"Test  :   {sum(test_list)} samples")
    print(f"Batch :   {sum(batch_list)} samples")
    
    # define and print source s3 locations
    #s3_images_source = f"s3://{default_bucket}/{prefix_data}/images/"
    #s3_labels_source = f"s3://{default_bucket}/{prefix_data}/labels/"
    s3_images_source = args.s3_images_source
    s3_labels_source = args.s3_labels_source
    print('Images source directory location:', s3_images_source)
    print('Labels source directory location:', s3_labels_source, '\n')

    # define and print destination s3 location for data splits
    s3_split_dest = args.s3_split_dest
    print('Split destination directory location:', s3_split_dest)
    
    # perform data copies
    print('Beginning TRAIN data split copies.')
    split_dataset(split_name='train', split_list=train_list)
    print('Completed TRAIN data split copies.\n')

    print('Beginning VAL data split copies.')
    split_dataset(split_name='val', split_list=val_list)
    print('Completed VAL data split copies.\n')

    print('Beginning TEST data split copies.')
    split_dataset(split_name='test', split_list=test_list)
    print('Completed TEST data split copies.\n')

    print('Beginning BATCH data split copies.')
    split_dataset(split_name='batch', split_list=batch_list)
    print('Completed BATCH data split copies.')

Overwriting code/preprocessing.py


In [12]:
from sagemaker.pytorch.processing import PyTorchProcessor

#Initialize the PyTorchProcessor
pytorch_processor = PyTorchProcessor(
    framework_version='2.1.0',
    role=role,
    instance_type='ml.m5.xlarge',
    instance_count=processing_instance_count,
    py_version='py310',
    #source_dir='code',
    base_job_name='pytorch-safety-process',
    sagemaker_session=pipeline_session,
)

In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = pytorch_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    arguments=["--s3-images-source", f"s3://{default_bucket}/{prefix_data}/images/",
                   "--s3-labels-source", f"s3://{default_bucket}/{prefix_data}/labels/",
                   "--s3-split-dest", f"s3://{default_bucket}/{prefix_data}/split_cicd/"
    ],
    code="preprocessing.py",
    source_dir='code',
)

step_process = ProcessingStep(name="SafetyProcess", step_args=processor_args)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


'\noutputs=[\n    ProcessingOutput(output_name="training", source=f"s3://{default_bucket}/{prefix_data}/split_cicd/"), #"/opt/ml/processing/training"),\n    #ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),\n    #ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),\n],\n'

## Define a Training Step to Train a Model

In [16]:
#from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

#model_path = f"s3://{default_bucket}/SafetyTrain"

# define job name and output location, 'bm' for benchmark
job_name = 'yolov8-cicd' # + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(default_bucket, prefix_data, job_name)

# build a PyTorch estimator
pytorch_estimator = PyTorch(
    role=role,
    entry_point='train.py', # custom training script, locate in code directory
    framework_version='2.1.0', # training - CPU - Python 3.10
    py_version='py310',
    source_dir='code',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=output_location,
    sagemaker_session=pipeline_session, # pipeline session
    hyperparameters = {'data': 'data.yaml', # yaml config file for custom dataset
                       'epochs': 1, # number of training epochs
                       'batch': 32, # batch size, -1 for AutoBatch
                       'yolo_model': 'yolov8n.pt', # pretrained base model
                       'saved_model_weights': 'model.pt' # saved model weights
                      }
)

# s3 location where training data is saved
# inputs = s3_split_dest[:-1]
inputs = f"s3://{default_bucket}/{prefix_data}/split_cicd"

# begin training job
train_args = pytorch_estimator.fit(inputs=inputs, job_name=job_name, logs='All')

In [17]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="SafetyTrain",
    step_args=train_args,
)

## Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including accuracy and ROC curve.
* Save the evaluation report to the evaluation directory.

In [18]:
%%writefile code/evaluation.py
import json
import pathlib
import pickle
import tarfile
import argparse
import joblib
import numpy as np
import pandas as pd
from ultralytics import YOLO


if __name__ == "__main__":
    
    parser = argparse.ArgumentParser()

    parser.add_argument('--data', type=str, default='data-eval.yaml') # yaml config file for custom dataset

    # obtain arguments
    args, _ = parser.parse_known_args()
    
    model_path = '/opt/ml/processing/model/model.tar.gz'
    with tarfile.open(model_path) as tar:
        tar.extractall(path="/opt/ml/processing/model/")

    #model = pickle.load(open("model.pt", "rb"))
    
    # evaluate model on test dataset
    print('EVALUATING MODEL ON TEST DATASET...')
    model_val = YOLO('/opt/ml/processing/model/model.pt')
    metrics = model_val.val(data=args.data, split='test')
    
    # print evaluation metric to compare performance between models
    print('-------------')
    print('-------------')
    print('-------------')
    print('MODEL EVALUATION METRIC:')
    print('mAP50:', round(metrics.box.map50, 4))
    print('-------------')
    print('-------------')
    print('-------------')
    
    
    map50 = round(metrics.box.map50, 4)

    report_dict = {
        "detection_metrics": {
            "mAP50": {"value": map50},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluation.py


Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

In [19]:
#Initialize the PyTorchProcessor
pytorch_processor_eval = PyTorchProcessor(
    framework_version='2.1.0',
    role=role,
    instance_type='ml.m5.xlarge',
    instance_count=processing_instance_count,
    py_version='py310',
    #source_dir='code',
    base_job_name='pytorch-safety-eval',
    sagemaker_session=pipeline_session,
)

eval_args = pytorch_processor_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=inputs, #step_process.properties.ProcessingOutputConfig.Outputs["training"].S3Output.S3Uri,
            destination="/opt/ml/processing/input/code/datasets",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="evaluation.py",
    source_dir='code',
)

In [20]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="SafetyEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

## Define a Create Model Step to Create a Model

In [95]:
# trained model artifacts
#model_data = step_train.properties.ModelArtifacts.S3ModelArtifacts
model_data = f"s3://sagemaker-us-east-1-414754026690/safety/data/output/yolov8-cicd/pipelines-kqrcqmtvskqa-SafetyTrain-yS7q405iW7/output/model.tar.gz"

# create a PyTorch model
pytorch_model = PyTorchModel(
    model_data=model_data, # trained model artifacts
    role=role,
    entry_point='inference.py', # custom inference script, locate in code directory
    framework_version='1.3',
    py_version='py3',
    source_dir='code',
    sagemaker_session=pipeline_session,
)

Define the `ModelStep` by providing the return values from `model.create()` as the step arguments.

In [96]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
#from sagemaker.model import Model # not sure if needed

step_create_model = ModelStep(
    name="SafetyCreateModel",
    step_args=pytorch_model.create(instance_type="ml.m5.xlarge", accelerator_type="ml.eia1.medium"),
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-414754026690/safety/data/output/yolov8-cicd/pipelines-kqrcqmtvskqa-SafetyTrain-yS7q405iW7/output/model.tar.gz), script artifact (code), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-414754026690/pytorch-inference-eia-2024-02-19-00-40-30-778/model.tar.gz. This may take some time depending on model size...


'\nstep_create_model = ModelStep(\n    name="SafetyCreateModel",\n    model=pytorch_model,\n    inputs=CreateModelInput(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),\n)\n'

## Define a Transform Step to Perform Batch Transformation

Now that a model instance is defined, create a `Transformer` instance with the appropriate model type, compute instance type, and desired output S3 URI.

Specifically, pass in the `ModelName` from the `CreateModelStep`, `step_create_model` properties. The `CreateModelStep` `properties` attribute matches the object model of the [DescribeModel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html) response object.

In [97]:
from sagemaker.transformer import Transformer

transformer_output = f"s3://{default_bucket}/SafetyTransform" # transformer job results

# create a transformer from the PyTorch model
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=transformer_output,
    accept='application/json',
    max_payload=10
)

Pass in the transformer instance and the `TransformInput` with the `batch_data` pipeline parameter defined earlier.

In [98]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="SafetyTransform", transformer=transformer, inputs=TransformInput(data=batch_data)
)

## Define a Register Model Step to Create a Model Package

A model package is an abstraction of reusable model artifacts that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

To register a model in the Model Registry, we take the model created in the previous steps
```
model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)
```
and call the `.register()` function on it while passing all the parameters needed for registering the model.

We take the outputs of the `.register()` call and pass that to the `ModelStep` as step arguments.

In [129]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

#eval_output = step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
#evaluation_s3_uri = f"{eval_output}/evaluation.json"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=f"s3://sagemaker-us-east-1-414754026690/SafetyPipeline/kqrcqmtvskqa/SafetyEval/output/evaluation/evaluation.json",
        #s3_uri="{}/evaluation.json".format(
        #    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        #),
        content_type="application/json"
    )
)

register_args = pytorch_model.register(
    content_types=["image/jpeg"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    domain="COMPUTER_VISION"
)

step_register = ModelStep(name="SafetyRegisterModel", step_args=register_args)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-414754026690/safety/data/output/yolov8-cicd/pipelines-kqrcqmtvskqa-SafetyTrain-yS7q405iW7/output/model.tar.gz), script artifact (code), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-414754026690/pytorch-inference-2024-02-19-01-46-49-736/model.tar.gz. This may take some time depending on model size...


## Define a Fail Step to Terminate the Pipeline Execution and Mark it as Failed

This section walks you through the following steps:

* Define a `FailStep` with customized error message, which indicates the cause of the execution failure.
* Enter the `FailStep` error message with a `Join` function, which appends a static text string with the dynamic `mse_threshold` parameter to build a more informative error message.

In [130]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="SafetyMAPFail",
    error_message=Join(on=" ", values=["Execution failed due to mAP <", map_threshold]),
)

## Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry, Or Terminate the Execution in Failed State

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. Otherwise, the pipeline execution fails and terminates. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties.

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed if the condition evaluates to `True`.
* Pass the `FailStep` step into the `else_steps`of the `ConditionStep`, which is only executed if the condition evaluates to `False`.

In [144]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="detection_metrics.mAP50.value",
    ),
    right=map_threshold,
)

step_cond = ConditionStep(
    name="SafetyMAPCond",
    conditions=[cond_gte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[step_fail],
)

## Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the data dependency DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

In [145]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"SafetyPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        map_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    #steps=[step_process, step_train, step_eval],
    #steps=[step_train, step_eval],
    #steps=[step_eval],
    #steps=[step_eval, step_cond]
)

### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and the parameters and step properties resolve correctly.

In [146]:
import json

definition = json.loads(pipeline.definition())
definition

INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/86b113c5543c262aefe08b88ae3a4e3d/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/88f4f1b66ade82e1fe99e11e78668722/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.r5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-414754026690/safety/catalog/catalog_query.csv'},
  {'Name': 'BatchData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-414754026690/safety/data/split_cicd/batch/images'},
  {'Name': 'mAPThreshold', 'Type': 'Float', 'DefaultValue': 0.001}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'SafetyProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {

## Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The Pipeline service uses the role that is passed in to create all the jobs defined in the steps.

In [147]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/86b113c5543c262aefe08b88ae3a4e3d/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/88f4f1b66ade82e1fe99e11e78668722/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/86b113c5543c262aefe08b88ae3a4e3d/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-414754026690/SafetyPipeline/code/54f0ef6bee583ff9186b762a

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:414754026690:pipeline/SafetyPipeline',
 'ResponseMetadata': {'RequestId': '490808ca-7358-4250-ab49-c24c948aeef7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '490808ca-7358-4250-ab49-c24c948aeef7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Mon, 19 Feb 2024 02:16:42 GMT'},
  'RetryAttempts': 0}}

Start the pipeline and accept all the default parameters.

In [148]:
execution = pipeline.start()

## Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [149]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:414754026690:pipeline/SafetyPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:414754026690:pipeline/SafetyPipeline/execution/karjsh3p6g92',
 'PipelineExecutionDisplayName': 'execution-1708309007967',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'safetypipeline',
  'TrialName': 'karjsh3p6g92'},
 'CreationTime': datetime.datetime(2024, 2, 19, 2, 16, 47, 834000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 2, 19, 2, 16, 47, 834000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:414754026690:user-profile/d-kv43uh6emydw/jraimondi',
  'UserProfileName': 'jraimondi',
  'DomainId': 'd-kv43uh6emydw'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:414754026690:user-profile/d-kv43uh6emydw/jraimondi',
  'UserProfileName': 'jraimondi',
  'DomainId': 'd-kv43uh6emydw'},
 'ResponseMetadata': {'RequestId': 'fe5b1c39-fc88-410c-8

Wait for the execution to complete.

In [150]:
execution.wait()

KeyboardInterrupt: 

List the steps in the execution. These are the steps in the pipeline that have been resolved by the step executor service.

In [155]:
execution.list_steps()

[{'StepName': 'SafetyTransform',
  'StartTime': datetime.datetime(2024, 2, 19, 2, 25, 36, 950000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 1,
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-1:414754026690:transform-job/pipelines-karjsh3p6g92-SafetyTransform-tqxkr0T9HB'}}},
 {'StepName': 'SafetyCreateModel-CreateModel',
  'StartTime': datetime.datetime(2024, 2, 19, 2, 25, 34, 898000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 19, 2, 25, 36, 440000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:414754026690:model/pipelines-karjsh3p6g92-safetycreatemodel-cr-wsbew7qi5i'}}},
 {'StepName': 'SafetyRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 2, 19, 2, 25, 34, 898000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 19, 2, 25, 36, 152000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata':

### Examining the Evaluation

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [ ]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

### Parametrized Executions

You can run additional executions of the pipeline and specify different pipeline parameters. The `parameters` argument is a dictionary containing parameter names, and where the values are used to override the defaults values.

Based on the performance of the model, you might want to kick off another pipeline execution on a compute-optimized instance type and set the model approval status to "Approved" automatically. This means that the model package version generated by the `RegisterModel` step is automatically ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

Apart from that, you might also want to adjust the MSE threshold to a smaller value and raise the bar for the accuracy of the registered model. In this case you can override the MSE threshold like the following:

In [ ]:
execution = pipeline.start(parameters=dict(MseThreshold=3.0))

If the MSE threshold is not satisfied, the pipeline execution enters the `FailStep` and is marked as failed.

In [ ]:
try:
    execution.wait()
except Exception as error:
    print(error)

In [ ]:
execution.list_steps()